In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
client = OpenAI(
    api_key = os.getenv('RUNPOD_TOKEN'),
    base_url= os.getenv('RUNPOD_CHATBOT_URL'),
    )
model_name = os.getenv("MODEL_NAME")

In [4]:
print(os.getenv('RUNPOD_TOKEN'))
print(os.getenv('RUNPOD_CHATBOT_URL'))
print(os.getenv('MODEL_NAME'))

rpa_C2WR2PF37Q107JHL44E2QJQ7SC8BF64EDQTAGXJE1t2re2
https://api.runpod.ai/v2/vllm-vi7necowx1y5o1/openai/v1
meta-llama/Meta-Llama-3-8B-Instruct


In [5]:
response = client.chat.completions.create(
    model = model_name,
    messages= [
        {"role": "user", "content ": "What is the capital of Germany?"}
    ],
    temperature= 0.0, # Index of the response
    top_p=0.8, # Probability of the response
    max_tokens = 2000 # Maximum number of tokens (words) in the response
)